# CS440 Assignment 1 - Search

## Part 1.1

## State Representation

In [78]:
class Node:
    def __init__(self, rowPos, colPos, value):
        self.rowPos = rowPos
        self.colPos = colPos
        self.value = value
        self.isWall = False  
        self.isGoal = False
        self.distance = 0

In [158]:
from collections import deque
from IPython.display import clear_output
import sys
import time

goal_dict = {}

class Maze:
    def __init__(self, filename):     
        self.goals = []
        self.boxes = []
        self.goalCounter = '0'
        self.start = None
        self.rowsInMaze = 0
        self.colsInMaze = 0
        self.colsList = []
        self.path = []
        self.explored = {}
        self.nodesExpanded = 0
        
        # Read maze from file
        mazeObject = open(filename, 'r')
        rows = 0
        
        # Loop through rows in Maze
        for line in mazeObject:
            rowList = []
            cols = 0
            
            # Loop through columns in Maze
            for char in line[:-1]:
                node = Node(rows, cols, char)
                
                rowList.append(node)
                
                # Look for starting point
                if char == 'P':
                    self.start = node
                    
                # Look for goals
                if char == '.':
                    node.isGoal = True
                    self.goals.append(node)
                    
                # Look for walls
                if char == '%':
                    node.isWall = True
                    
                # Look for boxes
                if char == 'b':
                    self.boxes.append(node)
                
                # Look for boxes
                if char == 'B':
                    self.boxes.append(node)
                    self.goals.append(node)
                    node.isGoal = True
                
                cols = cols + 1
            rows = rows + 1
        
            self.colsList.append(rowList)
            self.colsInMaze = len(rowList)
        
        self.rowsInMaze = rows
    
    def runSokobanSearch(self, searchType):
        startTime = time.time()
        state = State(self.start, self.goals[:], self.boxes[:])
        self.explored[hash(state)] = state
        
        # Initialize frontier based on starting node
        addToFrontier(state, self)
        
        while self.frontier != []:
            if searchType == 'bfs':
                nextState = self.frontier.popleft()
                state = sokobanMove(nextState, self)
        
            if searchType == 'astarv2': 
                index = 0
                total_distance = sys.maxsize
                for i in range(0, len(self.frontier)):
                    if self.frontier[i].distance <= total_distance:
                        total_distance = self.frontier[i].distance
                        index = i
                nextState = self.frontier.pop(index)
                state = sokobanMove(nextState, self)
            
            # Check if no goals exist
            if (sokobanGoalTest(state.goals)):
                endTime = time.time()
                print(endTime - startTime)
                path = []
                while state.previous != None:
                    path.append(state)
                    state = state.previous
                print("Nodes Expanded: " + str(self.nodesExpanded))
                print("Nodes in Path: " + str(len(path)))
                self.printMaze()
                return
    
    def runSearch(self, searchType):   
        startTime = time.time()
        state = State(self.start, self.goals[:], None)
        self.explored[hash(state)] = state
        
        if searchType == 'bfs':
            self.frontier = deque([])
        else:
            self.frontier = []

        # Initialize frontier based on starting node
        addToFrontier(state, self)        

        while self.frontier != []:
            # Search Algorithms
            if searchType == 'dfs':
                nextState = self.frontier.pop()
                state = move(nextState, self)
                
            if searchType == 'bfs':
                nextState = self.frontier.popleft()
                state = move(nextState, self)

            if searchType == 'gbfs':
                distance = sys.maxsize     
                index = 0
                for i in range(0, len(self.frontier)):
                    for goal in state.goals:
                        if abs(self.frontier[i].currentRowPos - goal.rowPos) + abs(self.frontier[i].currentColPos - goal.colPos) <= distance:
                            distance = abs(self.frontier[i].currentRowPos - goal.rowPos) + abs(self.frontier[i].currentColPos - goal.colPos)
                            index = i
                nextState = self.frontier.pop(index)
                state = move(nextState, self)

            if searchType == 'astar':
                distance = sys.maxsize     
                index = 0
                for i in range(0, len(self.frontier)):
                    for goal in state.goals:
                        distance_to_goal = abs(self.frontier[i].currentRowPos - goal.rowPos) + abs(self.frontier[i].currentColPos - goal.colPos)
                        distance_from_start = self.frontier[i].cost

                        if distance_to_goal + distance_from_start <= distance:
                            distance = distance_to_goal + distance_from_start
                            index = i

                nextState = self.frontier.pop(index)
                state = move(nextState, self)
                
            if searchType == 'astarv2': 
                index = 0
                total_distance = sys.maxsize
                for i in range(0, len(self.frontier)):
                    if self.frontier[i].distance <= total_distance:
                        total_distance = self.frontier[i].distance
                        index = i
                #tempValue = self.frontier[index].getCurrentNode(self).value
                #self.frontier[index].getCurrentNode(self).value = 'X'
                #self.printMaze()
                #self.frontier[index].getCurrentNode(self).value = tempValue
                print("Distance: " + str(self.frontier[index].distance))
                nextState = self.frontier.pop(index)
                state = move(nextState, self)

            # Check if no goals exist
            if (goalTest(state.goals)):
                endTime = time.time()
                print(endTime - startTime)
                path = []
                if len(self.goals) > 10:
                    self.goalCounter = chr(ord('a') + len(self.goals) - 10)
                else:
                    self.goalCounter = chr(ord('0') + len(self.goals))
                while state.previous != None:
                    if state.getCurrentNode(self).isGoal and state.getCurrentNode(self).value == '.':
                        state.getCurrentNode(self).value = self.goalCounter
                        self.goalCounter = incrementGoalCounter(self.goalCounter)
                    if state.getCurrentNode(self).value == ' ':
                        state.getCurrentNode(self).value = '.'
                    path.append(state)
                    state = state.previous
                print("Nodes Expanded: " + str(self.nodesExpanded))
                print("Nodes in Path: " + str(len(path)))
                self.printMaze()
                return

    def printMaze(self):          
        for row in self.colsList:
            line = ""
            for node in row:
                line += node.value
            print(line)

In [80]:
import sys
def calcDistance(start, goals):
    totalDistance = 0
    shortestGoal = None
    
    while goals != []:
        distance = sys.maxsize
        for goal in goals:
            if abs(start.rowPos - goal.rowPos) + abs(start.colPos - goal.colPos) <= distance:
                distance = abs(start.rowPos - goal.rowPos) + abs(start.colPos - goal.colPos)
                shortestGoal = goal
        goals.remove(shortestGoal)
        start = shortestGoal
        totalDistance += distance
    return totalDistance        

In [192]:
import sys
class Path():
    def __init__(self, currentGoal, remainingGoals):
        self.row = currentGoal.rowPos
        self.col = currentGoal.colPos
        self.remainingGoals = remainingGoals
        self.cost = 0
        
    def __hash__(self):
        return hash(str(self.row) + str(self.col) + hash_goals(self.remainingGoals))
    
def smallestPathCost(goal, remainingGoals):
    explored = {}
    if remainingGoals == []:
        return 0
    
    startPath = Path(goal, remainingGoals)
    explored[hash(startPath)] = startPath
    
    frontier = []
    
    for goal in startPath.remainingGoals:
        tempRemainingGoals = startPath.remainingGoals[:]
        tempRemainingGoals.remove(goal)
        newPath = Path(goal, tempRemainingGoals)
        newPath.cost = abs(startPath.row - newPath.row) + abs(startPath.col - newPath.col)
        frontier.append(newPath)
        
    while frontier != []:
        cost = sys.maxsize
        index = 0
        
        # Remove lowest cost path from frontier
        for i in range(len(frontier)):
            if frontier[i].cost <= cost:
                cost = frontier[i].cost
                index = i
        startPath = frontier.pop(index)
        if startPath in explored:
            pass
        else:
            #print(str(startPath.row) + "," + str(startPath.col) + " - " + str(startPath.cost))
        
            if startPath.remainingGoals == []:
                print(str(goal.rowPos) + "," + str(goal.colPos) + " - " + str(startPath.cost))
                return startPath.cost

            # Add to frontier
            for goal in startPath.remainingGoals:
                tempRemainingGoals = startPath.remainingGoals[:]
                tempRemainingGoals.remove(goal)
                newPath = Path(goal, tempRemainingGoals)
                newPath.cost = abs(startPath.row - newPath.row) + abs(startPath.col - newPath.col)  + startPath.cost
                frontier.append(newPath)
            #printedFrontier = []
            #for path in frontier:
                #printedFrontier.append(str(path.row) + "," + str(path.col) + " - " + str(path.cost))
            #print(printedFrontier)
            #input("Press enter to continue")

In [170]:
class State:
    def __init__(self, currentNode, goals, boxes):
        self.goals = goals
        self.boxes = boxes
        self.currentRowPos = currentNode.rowPos
        self.currentColPos = currentNode.colPos
        self.cost = 0
        self.previous = None
    
    def __hash__(self):
        goalString = ""
        boxString = ""
        for goal in self.goals:
            goalString += str(goal.rowPos) + str(goal.colPos)
        if self.boxes != None:
            for box in self.boxes:
                boxString += str(box.rowPos) + str(box.colPos)
        return hash((str(self.currentRowPos), str(self.currentColPos), goalString, boxString))
        
    def getCurrentNode(self, maze):
        return maze.colsList[self.currentRowPos][self.currentColPos]

## Transition Model

In [138]:
def hash_goals(goals):
    hash_string = ""
    for goal in goals:
        hash_string += str(goal.rowPos) + "," + str(goal.colPos)
    return hash_string

In [195]:
def move(state, maze):    
    # Check if state is goal
    if state.getCurrentNode(maze) in state.goals:
        tempGoals = state.goals[:]
        tempGoals.remove(state.getCurrentNode(maze))
        newState = State(state.getCurrentNode(maze), tempGoals[:], None)
        newState.previous = state
        newState.cost = state.cost
        distance = sys.maxsize
        for goal in newState.goals:
            tempGoals = state.goals[:]
            tempGoals.remove(goal)
            #distance_remaining = calcDistance(goal, tempGoals)
            distance_remaining = 0
            if (goal, hash_goals(tempGoals)) in goal_dict:
                distance_remaining = goal_dict[(goal, hash_goals(tempGoals))]
            else:
                distance_remaining = calcDistance(goal, tempGoals)
                goal_dict[(goal, hash_goals(tempGoals))] = distance_remaining
                
            distance_to_goal = abs(newState.currentRowPos - goal.rowPos) + abs(newState.currentColPos - goal.colPos)
            if distance_remaining + distance_to_goal + newState.cost <= distance:
                distance = distance_remaining + distance_to_goal + newState.cost         
        newState.distance = distance
        maze.explored[hash(newState)] = newState   
        addToFrontier(newState, maze)
        return newState
    elif hash(state) in maze.explored:
        return state
    else:   
        addToFrontier(state, maze)
        maze.explored[hash(state)] = state   
        return state
    
def sokobanMove(state, maze):
    if hash(state) in maze.explored:
        return state
    else:
        addToFrontier(state, maze)
        maze.explored[hash(state)] = state
        return state

In [196]:
from IPython.display import clear_output
import sys
def addToFrontier(state, maze): 
    maze.nodesExpanded += 1
    #print(maze.nodesExpanded)

    for goal in state.goals:
            tempGoals = state.goals[:]
            tempGoals.remove(goal)
            #distance_remaining.append(calcDistance(goal, tempGoals))
            distance_remaining = 0
            if (goal, hash_goals(tempGoals)) in goal_dict:
                distance_remaining = goal_dict[(goal, hash_goals(tempGoals))]
            else:
                distance_remaining = calcDistance(goal, tempGoals)
                goal_dict[(goal, hash_goals(tempGoals))] = distance_remaining
            
    
    # Top Node
    if state.currentRowPos > 1 and not maze.colsList[state.currentRowPos - 1][state.currentColPos].isWall:  
        newState = State(maze.colsList[state.currentRowPos - 1][state.currentColPos], state.goals[:], None)
        newState.cost = state.cost + 1
        newState.previous = state
        distance = sys.maxsize
        for i in range(0, len(newState.goals)):
            distance_to_goal = abs(newState.currentRowPos - newState.goals[i].rowPos) + abs(newState.currentColPos - newState.goals[i].colPos)
            if distance_remaining + distance_to_goal + newState.cost <= distance:
                distance = distance_remaining + distance_to_goal + newState.cost
                
        newState.distance = distance
        maze.frontier.append(newState)
        
    # Bottom Node
    if state.currentRowPos < maze.rowsInMaze - 1 and not maze.colsList[state.currentRowPos + 1][state.currentColPos].isWall:
        newState = State(maze.colsList[state.currentRowPos + 1][state.currentColPos], state.goals[:], None)
        newState.cost = state.cost + 1
        newState.previous = state
        distance = sys.maxsize
        for i in range(0, len(newState.goals)):
            distance_to_goal = abs(newState.currentRowPos - newState.goals[i].rowPos) + abs(newState.currentColPos - newState.goals[i].colPos)
            if distance_remaining + distance_to_goal + newState.cost <= distance:
                distance = distance_remaining + distance_to_goal + newState.cost
                
        newState.distance = distance
        maze.frontier.append(newState)
        
        
    # Left Node
    if state.currentRowPos > 1 and not maze.colsList[state.currentRowPos][state.currentColPos - 1].isWall:
        newState = State(maze.colsList[state.currentRowPos][state.currentColPos - 1], state.goals[:], None)
        newState.cost = state.cost + 1
        newState.previous = state
        distance = sys.maxsize
        for i in range(0, len(newState.goals)):
            distance_to_goal = abs(newState.currentRowPos - newState.goals[i].rowPos) + abs(newState.currentColPos - newState.goals[i].colPos)
            if distance_remaining + distance_to_goal + newState.cost <= distance:
                distance = distance_remaining + distance_to_goal + newState.cost
                
        newState.distance = distance
        maze.frontier.append(newState)
        
    # Right Node
    if state.currentColPos < maze.colsInMaze - 1 and not maze.colsList[state.currentRowPos][state.currentColPos + 1].isWall:
        newState = State(maze.colsList[state.currentRowPos][state.currentColPos + 1], state.goals[:], None)
        newState.cost = state.cost + 1
        newState.previous = state
        distance = sys.maxsize
        for i in range(0, len(newState.goals)):
            distance_to_goal = abs(newState.currentRowPos - newState.goals[i].rowPos) + abs(newState.currentColPos - newState.goals[i].colPos)
            if distance_remaining + distance_to_goal + newState.cost <= distance:
                distance = distance_remaining + distance_to_goal + newState.cost
                
        newState.distance = distance
        maze.frontier.append(newState)
        
    #printedFrontier = []
    #for state in maze.frontier:
        #printedFrontier.append(str(len(state.goals)))
    #print(printedFrontier)
    #maze.printMaze()
    ##input("Press enter to continue")

## Goal Test

In [25]:
def goalTest(goals):      
    if goals == []:
        return True
    else:
        return False

def sokobanGoalTest(goals):
    count = 0
    for goal in goals:
        if goal.value == 'B':
            count += 1
    if count == len(goals):
        return True
    else:
        return False

In [13]:
def incrementGoalCounter(counter):
    if counter == 'a':
        return '9'
    else:
        return chr(ord(counter) - 1)

## Tests

#### Medium Maze

##### Depth-first Search

In [96]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [97]:
mediumMaze.runSearch('dfs')

0.003001689910888672
Nodes Expanded: 167
Nodes in Path: 117
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %           %           %  ..1%
% %%%% %% % % %%%%% %%%%% %%% % %%%% %% %%% %% %% %%% % %.% %
% %     %   %     %       %   %       %     %   % % %   %.% %
% % %%% % %%%% %% %%%%%%%%% %%%%% %%% % %%% % % % % %%%%%.% %
% %   %   %       %       %       %   %   % % % %     %...% %
% % %%% %%% %%% %%% % %%% %%%% %% % %%% % %%% % %%% %%%.%%% %
%   %     %     %   % %         % % %   %     %   % %...% % %
% %%% %%% %%%%% %%%%% % %%% %%% %%%%% %%%%%% %%%% %%%.%%% % %
%     %       %         % % %         %...%     %  ...%   % %
% %%%%% %%%%% %%%%% %%% % % % %%%%%%%%%.%.% %%% %%%.%%% %%% %
% %   % %   %         % %   % %     %  .%.% %.......    %   %
% % %%% % %%% %%%%% % % %%%%% % % %%% %.%.%%%.%%%%% %%%%% %%%
%   %   % %       % % %   %   % %   % %.%.....%     %   % % %
% %%% %%% % % %%% %%% %%% % %%% %%% % %.%%%%%%%%% %%% % % % %
% %   %   

##### Breadth-first Search

In [98]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [99]:
mediumMaze.runSearch('bfs')

0.014511346817016602
Nodes Expanded: 609
Nodes in Path: 95
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %     ....  %...........%....1%
% %%%% %% % % %%%%% %%%%% %%% % %%%%.%%.%%%.%% %% %%%.%.% % %
% %     %   %     %       %   %  .... %.....%   % % %...% % %
% % %%% % %%%% %% %%%%%%%%% %%%%%.%%% % %%% % % % % %%%%% % %
% %   %   %       %       %   ....%   %   % % % %     %   % %
% % %%% %%% %%% %%% % %%% %%%%.%% % %%% % %%% % %%% %%% %%% %
%   %     %     %   % %........ % % %   %     %   % %   % % %
% %%% %%% %%%%% %%%%% %.%%% %%% %%%%% %%%%%% %%%% %%% %%% % %
%     %.......%    .....% % %         %   %     %     %   % %
% %%%%%.%%%%%.%%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %
% %   %.%   %.......  % %   % %     %   % % %           %   %
% % %%%.% %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%
%   %...% %       % % %   %   % %   % % %     %     %   % % %
% %%%.%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %
% %...%   %

##### Greedy Best-first Search

In [100]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [101]:
mediumMaze.runSearch('gbfs')

0.009005546569824219
Nodes Expanded: 104
Nodes in Path: 95
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %     ....  %...........%....1%
% %%%% %% % % %%%%% %%%%% %%% % %%%%.%%.%%%.%% %% %%%.%.% % %
% %     %   %     %       %   %  .... %.....%   % % %...% % %
% % %%% % %%%% %% %%%%%%%%% %%%%%.%%% % %%% % % % % %%%%% % %
% %   %   %       %       %   ....%   %   % % % %     %   % %
% % %%% %%% %%% %%% % %%% %%%%.%% % %%% % %%% % %%% %%% %%% %
%   %     %     %   % %........ % % %   %     %   % %   % % %
% %%% %%% %%%%% %%%%% %.%%% %%% %%%%% %%%%%% %%%% %%% %%% % %
%     %.......%    .....% % %         %   %     %     %   % %
% %%%%%.%%%%%.%%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %
% %   %.%   %.......  % %   % %     %   % % %           %   %
% % %%%.% %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%
%   %...% %       % % %   %   % %   % % %     %     %   % % %
% %%%.%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %
% %...%   %

##### A* Search

In [102]:
filename = "inputs/mediumMaze.txt"
mediumMaze = Maze(filename)

In [103]:
mediumMaze.runSearch('astar')

0.03002023696899414
Nodes Expanded: 332
Nodes in Path: 95
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%         % %           %     %           %...........%....1%
% %%%% %% % % %%%%% %%%%% %%% % %%%% %% %%%.%% %% %%%.%.% % %
% %     %   %     %       %   %       %.....%   % % %...% % %
% % %%% % %%%% %% %%%%%%%%% %%%%% %%% %.%%% % % % % %%%%% % %
% %   %   %       %       %       %   %.  % % % %     %   % %
% % %%% %%% %%% %%% % %%% %%%% %% % %%%.% %%% % %%% %%% %%% %
%   %     %     %   % %.........% % %...%     %   % %   % % %
% %%% %%% %%%%% %%%%% %.%%% %%%.%%%%%.%%%%%% %%%% %%% %%% % %
%     %.......%    .....% % %  .......%   %     %     %   % %
% %%%%%.%%%%%.%%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %
% %   %.%   %.......  % %   % %     %   % % %           %   %
% % %%%.% %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%
%   %...% %       % % %   %   % %   % % %     %     %   % % %
% %%%.%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %
% %...%   % 

#### Big Maze

##### Depth-first Search

In [104]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [105]:
bigMaze.runSearch('dfs')

0.007004499435424805
Nodes Expanded: 302
Nodes in Path: 241
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %.......%...% %     %   %   %   %    .........% %   %.........%.....%   %
%   %.%%%.% %%%.%.%.% % % %%% % % % % %%% %%%.%% %%%%.% % % %.% %%%%%.%.%%%.%%% %
%   %.....% % %...%.%   %   % % % % % %   % %.%     %.%   %  .%     %...% %...  %
%%% %%%% %% % %  %%.%% %% % %%% % % % % %%% %.%%%%% %.% %%%%%.%...% % %%% % %.% %
%   % %     %     %.....% %   %   % % %     %.....% %.%     %...%.......%   %.% %
%   % % % %   % % %.....% % % %%% % % %%%%% %%%%%.% %.%%%%% % % % %%% %.%%% %.%%%
% %   % % %   % % %.... %   %     % % ......%.....% %.....%...% %     %...%  ...%
% %%% % %%% % %%% % %..%% %%% %%% % %%.%%%%.%.%%%%% %.....%.%...%% %% %%%.% %%%.%
% %       % % %   % %...% %.....% % ... %  ...%      ...  %.%  .....% %...% %...%
% % %%%%% % % % %%% % %.%%%.%%%.% %%....% %%%%% %%% %%%.%%%.% %%%%%.%%%.%%% %.% %
%   %   %   % %     % %.....%  .......

##### Breadth-first Search

In [106]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [107]:
bigMaze.runSearch('bfs')

0.033022403717041016
Nodes Expanded: 1250
Nodes in Path: 149
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %.......%...% %     %   %   %   %             % %   %         %     %   %
%   %.%%%.% %%%.%.%.% % % %%% % % % % %%% %%% %% %%%% % % % % % %%%%% % %%% %%% %
%   %.....% % %...%.%   %   % % % % % %   % % %     % %   %   %     %   % %     %
%%% %%%% %% % %  %%.%% %% % %%% % % % % %%% % %%%%% % % %%%%% %   % % %%% % % % %
%   % %     %     %.    % %   %   % % %     %     % % %     %   %       %   % % %
%   % % % %   % % %.    % % % %%% % % %%%%% %%%%% % % %%%%% % % % %%% % %%% % %%%
% %   % % %   % % %...  %   %     % %       %     % %     %   % %     %   %     %
% %%% % %%% % %%% % %. %% %%% %%% % %% %%%% % %%%%% %     % %   %% %% %%% % %%% %
% %       % % %   % %...% %.....% %     %     %.........  % %       % %   % %   %
% % %%%%% % % % %%% % %.%%%.%%%.% %%    % %%%%%.%%% %%%.%%% % %%%%% %%% %%% % % %
%   %   %   % %     % %.....%...     

##### Greedy Best-first Search

In [108]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [109]:
bigMaze.runSearch('gbfs')

0.06104087829589844
Nodes Expanded: 288
Nodes in Path: 223
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %       %   % %     %   %   %   %             % %   %         %     %   %
%   %.%%% % %%% % % % % % %%% % % % % %%% %%% %% %%%% % % % % % %%%%% % %%% %%% %
%   %.... % % %   % %   %   % % % % % %   % % %     % %   %   %     %   % %     %
%%% %%%%.%% % %  %% %% %% % %%% % % % % %%% % %%%%% % % %%%%% %   % % %%% % % % %
%   % % ....%     %     % %   %   % % %     %     % % %     %   %       %   % % %
%   % % % %...% % %     % % % %%% % % %%%%% %%%%% % % %%%%% % % % %%% % %%% % %%%
% %   % % %  .% % %     %   %     % %       %     % %     %   % %     %   %     %
% %%% % %%% %.%%% % %  %% %%% %%% % %% %%%% % %%%%% %     % %   %% %% %%% % %%% %
% %       % %.%   % %   % %     % %     %     %           % %       % %   % %   %
% % %%%%% % %.% %%% % % %%% %%% % %%    % %%%%% %%% %%% %%% % %%%%% %%% %%% % % %
%   %   %   %.%     % %     %          

##### A* Search

In [110]:
filename = "inputs/bigMaze.txt"
bigMaze = Maze(filename)

In [111]:
bigMaze.runSearch('astar')

0.18562769889831543
Nodes Expanded: 1107
Nodes in Path: 149
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%P....  %.......%...% %     %   %   %   %             % %   %         %     %   %
%   %.%%%.% %%%.%.%.% % % %%% % % % % %%% %%% %% %%%% % % % % % %%%%% % %%% %%% %
%   %.....% % %...%.%   %   % % % % % %   % % %     % %   %   %     %   % %     %
%%% %%%% %% % %  %%.%% %% % %%% % % % % %%% % %%%%% % % %%%%% %   % % %%% % % % %
%   % %     %     %.... % %   %   % % %     %     % % %     %   %       %   % % %
%   % % % %   % % %   . % % % %%% % % %%%%% %%%%% % % %%%%% % % % %%% % %%% % %%%
% %   % % %   % % %   . %   %     % %       %     % %     %   % %     %   %     %
% %%% % %%% % %%% % % .%% %%% %%% % %% %%%% % %%%%% %     % %   %% %% %%% % %%% %
% %       % % %   % % ..% %.....% %     %     %.........  % %       % %   % %   %
% % %%%%% % % % %%% % %.%%%.%%%.% %%    % %%%%%.%%% %%%.%%% % %%%%% %%% %%% % % %
%   %   %   % %     % %.....%...      

#### Open Maze

##### Depth-first Search

In [112]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [113]:
openMaze.runSearch('dfs')

0.012008428573608398
Nodes Expanded: 385
Nodes in Path: 218
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P............%
%                     %.............%
%                     %.............%
%                     %.............%
%                     %.............%
%                     %%%%%%........%
%                          %........%
%                          %........%
%                          %........%
%..........................%........%
%........            ......%........%
%........%%%%%%%%%%%%...............%
%........%                          %
%........%                          %
%........%                          %
%........%                          %
%........%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


##### Breadth-first Search

In [114]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [115]:
openMaze.runSearch('bfs')

0.019012451171875
Nodes Expanded: 524
Nodes in Path: 46
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %......       %
%                     %%%%%%.       %
%                          %.       %
%                          %.       %
%                          %.       %
%                          %.       %
%       ...................%.       %
%       .%%%%%%%%%%%%     ...       %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


##### Greedy Best-first Search

In [116]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [117]:
openMaze.runSearch('gbfs')

0.03402256965637207
Nodes Expanded: 154
Nodes in Path: 46
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %......       %
%                     %%%%%%.       %
%                          %.       %
%                          %.       %
%                          %.       %
%                          %.       %
%       ..............     %.       %
%       .%%%%%%%%%%%%........       %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


##### A* Search

In [118]:
filename = "inputs/openMaze.txt"
openMaze = Maze(filename)

In [119]:
openMaze.runSearch('astar')

0.09456586837768555
Nodes Expanded: 220
Nodes in Path: 46
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P....        %
%                     %    .        %
%                     %    .        %
%                     %    .        %
%                     %    ..       %
%                     %%%%%%.       %
%                          %.       %
%                          %.       %
%                          %.       %
%                          %.       %
%       ..............     %.       %
%       .%%%%%%%%%%%%........       %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .%                          %
%       .1%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


## Part 1.2

#### Tiny Search

##### Breadth-first Search

In [153]:
filename = "inputs/tinySearch.txt"
tinySearch = Maze(filename)

In [154]:
tinySearch.runSearch('bfs')

11.41011667251587
Nodes Expanded: 43527
Nodes in Path: 51
%%%%%%%%%%
%b..%.3..%
%.%c%.%%.%
%.% ..2%4%
%.a%P%  .%
%9  1 .6.%
%.%%%%.%.%
%8.7...%5%
%%%%%%%%%



##### A* Search

In [197]:
filename = "inputs/tinySearch.txt"
tinySearch = Maze(filename)

In [198]:
tinySearch.runSearch('astarv2')

Distance: 31
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 34
Distance: 34
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 34
Distance: 34
Distance: 34
Distance: 34
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 31
Distance: 31
Distance: 31
Distance: 31
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 33
Distance: 34
Distance: 34
Distance: 34
Distance: 34
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35

Distance: 37
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 35
Distance: 36
Distance: 36
Distance: 36
Distance: 36
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 36
Distance: 36
Distance: 36
Distance: 36
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 37
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 38
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39

Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40

Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 39
Distance: 39
Distance: 39
Distance: 39
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41

Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 39
Distance: 39
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41

Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42

Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 39
Distance: 39
Distance: 39
Distance: 38
Distance: 39
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 40
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 41
Distance: 40
Distance: 41
Distance: 41
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42
Distance: 42

#### Small Search

##### A* Search

In [199]:
filename = "inputs/smallSearch.txt"
smallSearch = Maze(filename)

In [200]:
smallSearch.runSearch('astarv2')

Distance: 78
Distance: 79
Distance: 79
Distance: 79
Distance: 79
Distance: 79
Distance: 79
Distance: 80
Distance: 80
Distance: 80
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 81
Distance: 82
Distance: 82
Distance: 82
Distance: 82
Distance: 82
Distance: 82
Distance: 82
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 83
Distance: 85
Distance: 85
Distance: 85
Distance: 85
Distance: 85
Distance: 85
Distance: 85
Distance: 86
Distance: 86
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87
Distance: 87

Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 96
Distance: 91
Distance: 93
Distance: 96
Distance: 96

Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distance: 101
Distan

Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distance: 103
Distan

Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distance: 105
Distan

...

Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 125
Distance: 125
Distance: 125
Distance: 125
Distance: 125
Distance: 125
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 126
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distan

Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distan

Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distan

Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distance: 127
Distan

KeyboardInterrupt: 

#### Medium Search

##### A* Search

In [118]:
filename = "inputs/mediumSearch.txt"
mediumSearch = Maze(filename)

In [ ]:
mediumSearch.runSearch('astarv2')

# Part 2

In [20]:
filename = "inputs/input1.txt"
input1 = Maze(filename)

In [21]:
input1.printMaze()

%%%%%%
%    %
% %  %
%Bb .%
%P%%%%
% %%%%
%%%%%
